In [5]:
import import_ipynb

import utils
from utils import *

In [6]:
# joint.ipynb (EWC 기반 Joint 학습)

import numpy as np
import pandas as pd
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers.legacy import Adamax
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical

In [8]:
# 데이터 불러오기
def load_mon_data():
    data_a = pd.read_pickle('mon_data_A.pkl')
    data_b = pd.read_pickle('mon_data_B.pkl')
    return data_a, data_b


# 데이터 로드 및 전처리
data_a, data_b = load_mon_data()

# mon_data_A를 train, test로 분리
first_part = split_by_label(data_a, 0, 49)
train_A, test_A = split_train_test(first_part, test_size=0.2, random_state=11)

# 3D ndarray 로 변환 
train_seq_A, train_label_A = split_data_label(train_A)

train_seq_A = np.stack(train_seq_A.values)
train_seq_A = train_seq_A[..., np.newaxis]

train_label_A = train_label_A.values
train_label_A = to_categorical(train_label_A, num_classes=95)

# mon_data_B를 train, test로 분리
second_part = split_by_label(data_b, 50, 94)
train_B, test_B = split_train_test(second_part, test_size=0.2, random_state=11)

#   3D ndarray 로 변환 
train_seq_B, train_label_B = split_data_label(train_B)

train_seq_B = np.stack(train_seq_B.values)
train_seq_B = train_seq_B[..., np.newaxis]

train_label_B = train_label_B.values
train_label_B = to_categorical(train_label_B, num_classes=95)

# mon_data_A의 train과 mon_data_B의 train 합치기
def accumulate(data1, data2):
    return np.concatenate([data1, data2], axis=0)
train_seq = accumulate(train_seq_A, train_seq_B)
train_label = accumulate(train_label_A, train_label_B)

In [10]:
# 3D ndarray 로 변환 
test_seq_A, test_label_A = split_data_label(test_A)

test_seq_A = np.stack(test_seq_A.values)
test_seq_A = test_seq_A[..., np.newaxis]

test_label_A = test_label_A.values
test_label_A = to_categorical(test_label_A, num_classes=95)

#   3D ndarray 로 변환 
test_seq_B, test_label_B = split_data_label(test_B)

test_seq_B = np.stack(test_seq_B.values)
test_seq_B = test_seq_B[..., np.newaxis]

test_label_B = test_label_B.values
test_label_B = to_categorical(test_label_B, num_classes=95)

# mon_data_A의 test와 mon_data_B의 test 합치기
test_seq = accumulate(test_seq_A, test_seq_B)
test_label = accumulate(test_label_A, test_label_B)

In [12]:
# 설정
num_sample = 1000
epochs=30
MAX_LABEL =  95


# joint_A에서 학습한 모델 및 정보 불러오기
model = load_model("joint_model.h5")

# mon_data_A의 train과 mon_data_B의 train으로 학습 수행
#train_seq, train_label = to_input(data, MAX_LABEL)

# EWC 손실로 모델 재컴파일
OPTIMIZER = Adamax(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss="categorical_crossentropy", optimizer=OPTIMIZER,
	metrics=["accuracy"])

# 학습
model.fit(train_seq, train_label,
          epochs=epochs,
          verbose=2)

# 평가: mon_data test셋 전체
loss, acc = model.evaluate(test_seq, test_label, verbose=0)
print(f"[Joint EWC] Accuracy on A+B after training on B: {acc:.4f}")


Epoch 1/30
475/475 - 38s - loss: 4.8452 - accuracy: 0.4161 - 38s/epoch - 80ms/step
Epoch 2/30
475/475 - 27s - loss: 3.1246 - accuracy: 0.4252 - 27s/epoch - 58ms/step
Epoch 3/30
475/475 - 27s - loss: 2.8070 - accuracy: 0.4470 - 27s/epoch - 57ms/step
Epoch 4/30
475/475 - 27s - loss: 2.5721 - accuracy: 0.4635 - 27s/epoch - 58ms/step
Epoch 5/30
475/475 - 28s - loss: 2.3291 - accuracy: 0.4911 - 28s/epoch - 58ms/step
Epoch 6/30
475/475 - 27s - loss: 2.0635 - accuracy: 0.5341 - 27s/epoch - 58ms/step
Epoch 7/30
475/475 - 27s - loss: 1.8226 - accuracy: 0.5808 - 27s/epoch - 58ms/step
Epoch 8/30
475/475 - 27s - loss: 1.5727 - accuracy: 0.6311 - 27s/epoch - 58ms/step
Epoch 9/30
475/475 - 28s - loss: 1.3874 - accuracy: 0.6679 - 28s/epoch - 58ms/step
Epoch 10/30
475/475 - 28s - loss: 1.2304 - accuracy: 0.7040 - 28s/epoch - 59ms/step
Epoch 11/30
475/475 - 28s - loss: 1.1016 - accuracy: 0.7343 - 28s/epoch - 58ms/step
Epoch 12/30
475/475 - 28s - loss: 0.9912 - accuracy: 0.7526 - 28s/epoch - 58ms/step
E